In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
products = pd.read_csv('./data/product_info.csv')
products.head(2)

,name,brand,category,price,ingredients,no_reviews,hearts,size1,size2,url
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,moisturizing-cream-oils-mists,$68.00,"<div aria-labelledby=""tab2"" class=""css-1rny024...",3K reviews,216935,SIZE 1.69 oz/ 50 mL•ITEM 2025633,0,https://www.sephora.com/product/protini-tm-pol...
1,The Water Cream,Tatcha,moisturizing-cream-oils-mists,$68.00,"<div aria-labelledby=""tab2"" class=""css-1rny024...",2K reviews,197492,ITEM 1932920,SIZE: 1.7 oz/ 50 mL,https://www.sephora.com/product/the-water-crea...


In [3]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2765 entries, 0 to 2764
Data columns (total 10 columns):
name           2765 non-null object
brand          2765 non-null object
category       2765 non-null object
price          2765 non-null object
ingredients    2754 non-null object
no_reviews     2765 non-null object
hearts         2765 non-null int64
size1          2765 non-null object
size2          2765 non-null object
url            2765 non-null object
dtypes: int64(1), object(9)
memory usage: 216.1+ KB


In [4]:
products[products.name == 'Pure One Step Camellia Oil Cleanser']

,name,brand,category,price,ingredients,no_reviews,hearts,size1,size2,url
34,Pure One Step Camellia Oil Cleanser,Tatcha,moisturizing-cream-oils-mists,$48.00,"<div aria-labelledby=""tab2"" class=""css-1rny024...",1K reviews,88702,ITEM 1673805,SIZE: 5.1 oz/ 150 mL,https://www.sephora.com/product/pure-one-step-...
841,Pure One Step Camellia Oil Cleanser,Tatcha,cleanser,$48.00,"<div aria-labelledby=""tab2"" class=""css-1rny024...",1K reviews,88740,ITEM 1673805,SIZE: 5.1 oz/ 150 mL,https://www.sephora.com/product/pure-one-step-...


In [5]:
#dropping duplicate occurance by 'name' and 'brand' of product
products.drop_duplicates(['name', 'brand'], keep="first", inplace = True)

In [6]:
#converting the price to float
products.price = products.price.str.replace("$", "")
# this format for some: '$185.00($214.00 value)'
for i in products.index:
    # this format for some: '$185.00($214.00 value)'
    products.loc[i, 'price'] =  products.price[i].split("(")[0]
    #this format for some: $4.00 $2.00
    products.loc[i, "price"] =  products.price[i].split(" ")[0]
#setting it as float
products.price = products.price.astype(float)

In [7]:
#dropping any products that ingredients were unable to be gathered
products = products[products.ingredients.notnull()]

In [8]:
# Take onle the part with the real ingredients 
pattern = ['<b>','<br>', '-\w+: ', 'Please', 'No Info', 
           'This product','Visit', 'Helpes', 'Serves', 'plays', 'Plays',
          'Acts', 'acts', 'Clean at Sephora products are', 'the synthetic fragrances',
           "Please be aware that ingredient lists may change", "Supports", "Reduces", "Restores", 
          "easily", "moisture", 'moisturizes', 'nourishes', 'penetrate', 'Help', 'reduce', 'improves', 'Hydrates',
          'appearance', "improving", 'improve', 'benefits', 'neutrilizes', 'formula', 'original', 'natural',
          'nurture', 'moisturize', 'moisturizes', 'protects', 'soothes', 'promote', 'regenerate',
          'protecting', 'updated periodically', 'addressing', 'nourish', 'Nurture', 'help', 'supports', 'Soothes',
          'Protects', 'innovative technology', 'helps', 'subject to change']

products.loc[:, 'ingredients'] = products.ingredients.str.split('<br/>')

In [9]:
#looping through all the rows
for i in products.index:
   #looping through the elemnts in the row's list
    for j in range(len(products.ingredients[i])):
        #if non of the words in pattern present in the string then it's an ingredient list
        if all(x not in products.ingredients[i][j] for x in pattern) and (products.ingredients[i][j].strip()):
            ingredient = products.ingredients[i][j]
    products.loc[i, 'ingredients'] = ingredient

   

In [10]:
#removing noise from ingredients
remove1 = ".\r\n"
remove2 = "</div></div>"
remove3 = '<div aria-labelledby="tab2" class="css-1rny024" hidden="" id="tabpanel2" role="tabpanel" tabindex="0"><div class="css-pz80c5">'
remove4 = '\r\r'
remove5 = '\n'
remove6 = '<div arialabelledby="tab2" class="css1rny024" hidden="" id="tabpanel2" role="tabpanel" tabindex="0"><p class="css189uj0g" datacomp="Text Box ">Get more information about<! > </p><div class="cssk13wt5" datacomp="Link Box ">shipping rates'


products['ingredients'] = products['ingredients'].str.replace(remove1, "")
products['ingredients'] = products['ingredients'].str.replace(remove2, "")
products['ingredients'] = products['ingredients'].str.replace(remove3, "")
products['ingredients'] = products['ingredients'].str.replace(remove4, "")
products['ingredients'] = products['ingredients'].str.replace(remove5, "")
products['ingredients'] = products['ingredients'].str.replace(remove6, "")

products['ingredients'] = products['ingredients'].str.replace('-', "")

#strip leading and trailing white spaces

products['ingredients'] = products['ingredients'].str.strip()

In [11]:
#I decided to remove the "water" ingredient instances because it's mostly just a carrier agent 
#it appears in almost every row and would distort the vectorizing and put too much weight on it

water = ['Water, ', 'Water/Aqua/Eau, ', 'Aqua/Water, ', 'Aqua (Water), ', 'Water , ', 'Aqua/Water/Eau, ',
        'Water (Aqua, Eau), ', 'Water/Aqua/Eau (Aqua), ', ' Water, ', 'Water/Eau, ']

for i in range(len(water)):
    products['ingredients'] = products['ingredients'].str.replace(water[i], "")



In [12]:
#checking which products do not have an ingredints list and have an empty string instead
for i in products.index:
    if not products.ingredients[i].strip():
        print(i)

45
139
349
1174
1175
1839
2118


In [13]:
#manually filling in the ingredients that failed to fill
#checkin what is their names
products.loc[[45, 139, 349, 1174, 1175, 1839, 2118], 'name']

45      BB Tinted Treatment 12-Hour Primer Broad Spect...
139     Bio-Performance Advanced Super Revitalizing Cream
349                   Regenerative Anti-Aging Moisturizer
1174    Purity Made Simple® Facial Cleansing Gel & Eye...
1175    EAU FRAÎCHE DOUCEUR Micellar Cleansing Water F...
1839       The Microdelivery Triple-Acid Brightening Peel
2118                                      Eye Contour Gel
Name: name, dtype: object

In [14]:
products.loc[45, 'ingredients'] = "Cyclopentasiloxane, Isododecane, Mica, Polysilicone-11, Polymethylsilsesquioxane, Hexyl Laurate, PEG-10 Dimethicone, Polyglyceryl-4 Isostearate, Stearic Acid, Cetyl PEG/PPG-10/1 Dimethicone, Alumina, Triethoxycaprylylsilane, Dipalmitoyl Hydroxyproline, Diamond Powder, Iron Oxides"
products.loc[139, 'ingredients'] = "Glycerin, Cyclomethicone, Butylene Glycol, Dimethicone, Cetyl Octanoate, Squalane, Dimethicone Copolyol, Disteardimonium Hectorite, Hydrogenated C6-14 Olefin Polymers, PEG-150, Mortierella Oil, Phytosteryl Macadamiate, Tocopheryl Acetate, Stearyl Glycyrrhetinate, Arginine HCI, PEG-60 Hydrogenated Castor Oil, Sodium Glutamate, Disodium Adenosine Triphosphate, Saccharomyces Lysate Extract, Sodium Acetylated Hyaluronate, Rosa Roxburghii Extract, Octyl Methoxycinnamate, Agar, Sodium Hexametaphosphate, Trisodium EDTA, Tocopherol, BHT, Ethylparaben, Butylparaben, Methylparaben, Fragrance, Iron Oxides"
products.loc[349, 'ingredients'] = "Caprylic/Capric Triglyceride, Butylene Glycol, Dimethicone, Pentylene Glycol, Butyrospermum Parkii (Shea) Butter, Sorbitan Stearate, Glycerin, Glyceryl Stearate, PEG-100 Stearate, Limnanthes Alba (Meadowfoam) Seed Oil, DI-C12-15 Alkyl Fumarate, Algae Exopolysaccharides, Malus Domestica Fruit (Apple) Cell Culture Extract, Bambusa Vulgaris (Bamboo) Leaf/Stem Extract, Tetrapeptide-21, Pisum Sativum (Pea) Extract, Enantia Chlorantha Bark Extract, Alaria Esculenta Extract, Sodium Ascorbyl Phosphate, Biotin, Glucosamine HCL, Eucalyptus Globulus Leaf Oil, Ergothioneine, Oleanolic Acid, Cetyl Alcohol, Dimethicone/PEG-10/15 Crosspolymer, Stearic Acid, Carbomer, Xanthan Gum, Lecithin, Mannitol, Vinyl Dimethicone/Methicone Silsesquioxane Crosspolymer, Disodium EDTA, Caprylyl Glycol, Ethylhexylglycerin, Aminomethyl Propanol, Hexylene Glycol, Phenoxyethanol"
products.loc[1174, 'ingredients'] = "Sodium Trideceth Sulfate, Disodium Lauroamphodiacetate, Acrylates Copolymer, Polysorbate 20, Sodium C14-16 Olefin Sulfonate, Glycerin, Cocamidopropyl Betaine, Isopropyl Alcohol, Sodium Sulfate, Limnanthes Alba (Meadowfoam) Seed Oil, Aniba Rosaeodora (Rosewood) Wood Oil, Pelargonium Graveolens Flower Oil, Bulnesia Sarmientoi Wood Oil, Cymbopogon Martini Oil, Rosa Centifolia Flower Oil, Amyris Balsamifera Bark Oil, Santalum Album (Sandalwood) Oil, Salvia Sclarea (Clary) Oil, Ormenis Multicaulis Oil, Acacia Dealbata Flower/Stem Extract, Daucus Carota Sativa (Carrot) Seed Oil, Piper Nigrum (Pepper) Fruit Oil, Disteareth-75 Ipdi, Glycereth-7 Caprylate/Caprate, Potassium Chloride, Hydrogen Peroxide, Magnesium Nitrate, Magnesium Chloride, Sodium Benzotriazolyl Butylphenol Sulfonate, Buteth-3, Tributyl Citrate, Sodium Hydroxide, Sodium Chloride, Disodium Edta, Citric Acid, Linalool, Methylchloroisothiazolinone, Methylisothiazolinone"
products.loc[1175, 'ingredients'] = "Hexylene Glycol, Glycerin, Poloxamer 184, Dihydrocholeth-30, Polyaminopropyl Biguanide, Benzyl Salicylate, Propylene Glycol, Fragrance, Disodium Cocoamphodiacetate, Disodium Edta, Rosa Gallica Extract/ Rosa Gallica Flower Extract"
products.loc[1839, 'ingredients'] = "Sd Alcohol 40b (Alcohol Denat.), Glycereth-7 Trimethyl Ether, Niacinamide, Mandelic Acid, Peg-8/Smdi Copolymer, Triethanolamine, Azelaic Acid, Phytic Acid, Bisabolol"
products.loc[2118, 'ingredients'] = "Rosa Damascena Flower Water, Citrus Aurantium Amara (bitter Orange) Flower Water, Butylene Glycol, Glycerin, Carbomer, Tromethamine, Caffeine, Phenoxyethanol, Ethylhexylglycerin, PEG-8, Aloe Barbadensis Leaf Juice, PEG-32, Citric Acid, Benzyl Acohol, Centaurea Cyanus Flower Extract, Sodium Benzoate, Ginkgo Biloba Leaf Extract, Potassium Sorbate, Methylisothiazolinone, Chamomilla Recutita (Matricaria) Flower Extract, Dehydroacetic Acid, CI 42090/Blue 1"



In [15]:
#converting # reviews to integer
products['no_reviews'] = products['no_reviews'].str.replace(" reviews", "").str.replace(".review", "").str.replace("K", "000").astype(int)

In [16]:
#this extract oz if those exist otherwise sets it to 0
for i in products.index:
    
    split = products.size1[i].split("/")[0]
    results = re.search("SIZE (.*) oz", split)
    try:
        products.loc[i, 'size1'] = results.group(1)
    except:
        products.loc[i, 'size1'] = '0'

    
#size one has a size if size 1 doesn't
for i in products.index:
    
    split = products.size2[i].split("/")[0]
    results = re.search("SIZE: (.*) oz", split)
    try:
        products.loc[i, 'size2'] = results.group(1)
    except:
        products.loc[i, 'size2'] = '0'

In [17]:
        
#remove " fl" and 'Limited Edition' and the like from the number of oz
products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace(" fl", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace(" fl", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("fl", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("fl", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("Limited Edition ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("Limited Edition ", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("2 x ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("2 x ", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("3 x ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("3 x ", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("Balm ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("Balm ", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("Standard Size - ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("Standard Size - ", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("28 x 0.01", '0.28')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("28 x 0.01", '0.28')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("30 x 0.06", '1.8')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("30 x 0.06", '1.8')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("90 x 0.012", '1.08')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("90 x 0.012", '1.08')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("50 x 0.04", '2')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("50 x 0.04", '2')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("7 Ampoules, .0625", '0.4375')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("7 Ampoules, .0625", '0.4375')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("4 Count - 0.29", '1.16')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("4 Count - 0.29", '1.16')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("4 x 1.25", '5')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("4 x 1.25", '5')

products.loc[2038, 'size1'] = 0
products.loc[37, 'size2'] = 1.7
products = products[products.name != 'Cleansing Spa Water Cloths'] #these are cleansing cloths not liquid products

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("Single-use mask ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("Single-use mask ", '')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("12 packettes 0.02", '0.24')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("12 packettes 0.02", '0.24')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("7 Sachets; 0.33", '2.31')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("7 Sachets; 0.33", '2.31')

products.loc[:, 'size1'] = products.loc[:, 'size1'].str.replace("Lip Crayon: ", '')
products.loc[:, 'size2'] = products.loc[:, 'size2'].str.replace("Lip Crayon: ", '')




#converting the size1 and size2 to floats
products.loc[:, ['size1', 'size2']] = products.loc[:, ['size1', 'size2']].astype(float)


#make a column that has the size for the product from size1 + size2
products.loc[:, 'final_size'] = products.size1 + products.size2

In [18]:
products.head(2)

,name,brand,category,price,ingredients,no_reviews,hearts,size1,size2,url,final_size
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,moisturizing-cream-oils-mists,68.0,"Dicaprylyl Carbonate, Glycerin, Cetearyl Alcoh...",3000,216935,1.69,0.0,https://www.sephora.com/product/protini-tm-pol...,1.69
1,The Water Cream,Tatcha,moisturizing-cream-oils-mists,68.0,Saccharomyces/Camellia Sinensis Leaf/Cladosiph...,2000,197492,0.00,1.7,https://www.sephora.com/product/the-water-crea...,1.70


In [19]:
#imputing 0 final_sizes by mean of the category
products['final_size'] = products['final_size'].replace(0, np.NaN)
products.loc[:, 'final_size'] = products.groupby(['category'])['final_size'].apply(lambda x:x.fillna(x.mean()))


In [20]:
#creating a price per ounce columns
products.loc[:, 'price_per_ounce'] = products['price'] / products['final_size']

In [21]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2452 entries, 0 to 2764
Data columns (total 12 columns):
name               2452 non-null object
brand              2452 non-null object
category           2452 non-null object
price              2452 non-null float64
ingredients        2452 non-null object
no_reviews         2452 non-null int64
hearts             2452 non-null int64
size1              2451 non-null float64
size2              2451 non-null float64
url                2452 non-null object
final_size         2452 non-null float64
price_per_ounce    2452 non-null float64
dtypes: float64(5), int64(2), object(5)
memory usage: 249.0+ KB


In [22]:
products.head(10)

,name,brand,category,price,ingredients,no_reviews,hearts,size1,size2,url,final_size,price_per_ounce
0,Protini™ Polypeptide Moisturizer,Drunk Elephant,moisturizing-cream-oils-mists,68.0,"Dicaprylyl Carbonate, Glycerin, Cetearyl Alcoh...",3000,216935,1.69,0.00,https://www.sephora.com/product/protini-tm-pol...,1.69,40.236686
1,The Water Cream,Tatcha,moisturizing-cream-oils-mists,68.0,Saccharomyces/Camellia Sinensis Leaf/Cladosiph...,2000,197492,0.00,1.70,https://www.sephora.com/product/the-water-crea...,1.70,40.000000
2,Ultra Facial Cream,Kiehl's Since 1851,moisturizing-cream-oils-mists,32.0,"Aqua/Glycerin, Cyclohexasiloxane, Squalane, Bi...",943,87617,0.00,1.70,https://www.sephora.com/product/ultra-facial-c...,1.70,18.823529
3,CC+ Cream with SPF 50+,IT Cosmetics,moisturizing-cream-oils-mists,39.5,"Snail Secretion Filtrate, Phenyl Trimethicone,...",3000,225410,1.08,0.00,https://www.sephora.com/product/your-skin-but-...,1.08,36.574074
4,The Dewy Skin Cream,Tatcha,moisturizing-cream-oils-mists,68.0,Saccharomyces/Camellia Sinensis Leaf/Cladosiph...,1000,85005,0.00,1.70,https://www.sephora.com/product/the-dewy-skin-...,1.70,40.000000
5,Lala Retro™ Whipped Moisturizer with Ceramides,Drunk Elephant,moisturizing-cream-oils-mists,60.0,"Glycerin, Caprylic/ Capric Triglyceride, Isopr...",909,40470,1.69,0.00,https://www.sephora.com/product/drunk-elephant...,1.69,35.502959
6,Crème de la Mer Moisturizer,La Mer,moisturizing-cream-oils-mists,180.0,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",612,72214,0.00,1.00,https://www.sephora.com/product/creme-de-la-me...,1.00,180.000000
7,F-Balm™ Electrolyte Waterfacial Mask,Drunk Elephant,moisturizing-cream-oils-mists,52.0,"Squalane, Propanediol, Niacinamide, Olive Oil ...",478,31337,1.69,0.00,https://www.sephora.com/product/drunk-elephant...,1.69,30.769231
8,The True Cream Aqua Bomb,belif,moisturizing-cream-oils-mists,38.0,"Dipropylene Glycol, Glycerin, Methl Trimethico...",3000,173760,0.00,1.68,https://www.sephora.com/product/the-true-cream...,1.68,22.619048
9,Virgin Marula Antioxidant Face Oil,Drunk Elephant,moisturizing-cream-oils-mists,72.0,100% Unrefined Sclerocraya Birrea (Marula) Ker...,1000,143287,0.00,1.00,https://www.sephora.com/product/virgin-marula-...,1.00,72.000000


In [23]:
products.to_csv("./data/products_clean.csv", index = False)